##   Code for running 4DVAR with the Lorenz 63 model.
####      Developed by Aneesh Subramanian, Ryan Torn, Greg Hakim.

In [1]:
import numpy as np
import time
from numpy.linalg import inv
import netCDF4 as nc
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import lorenz63_model as lor

In [2]:
assim_len = 1.0
fcst_len  = 2.0
nassim    = 50
alpha     = 4e-3
maxiters  = 50     # maximum number of iterations over the minimization scheme
tol       = 1e-2   # tolerance to end variational iteration
fdvarwin  = 0.35  # length of 4dvar iteration time period 
toff      = 0.75  # time offset: forecast from analysis to background time
nobstimes = 8   # number of evenly space obs times during fdvarwin
tlmdt     = 0.001  #  Time step for tangent linear model

In [3]:
H = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
#H = np.array([[1., 0., 0. ]]) #  observation operator for single observation
nobs = len(H[:,0])
R = np.eye(3) * 1.0e-2
#R = np.array([[1.0e-2]])  #  observation error for single observation

In [4]:
time1 = time.time()

In [5]:
np.random.seed(0)

In [8]:
ctime = toff + fdvarwin - assim_len
if ctime < 0:
  print('assimilation window is too short')

tshift = toff-ctime

odt = fdvarwin/float(nobstimes-1)

ntlm = int(round(odt / tlmdt))

bfile = nc.Dataset('L63_B.nc')
B = bfile.variables['B_matrix'][:,:]

invB = inv(B)
invR = inv(R)

# IC for truth taken from last time (column vector):
xt = np.array(lor.advance(10., 20., 30., 100.))

# populate initial state by perturbing true state
xa = xt + np.random.normal(0, 0.1, 3)
xb = np.empty(3)
xf = np.empty(3)
xe = np.empty(3)
xo = np.empty(3)
xi = np.empty(3)
xobs = np.empty(3)

#  Create error arrays
xbver = np.empty(3)
xaerr = np.empty((nassim, 3))
xberr = np.empty((nassim, 3))
xferr = np.empty((nassim, 3))

#  Create observation arrays
innov   = np.empty(nobs)
yobs    = np.empty((nobs, nobstimes))

#  Create TLM arrays
ttraj   = np.empty(nobstimes*ntlm+1)
xtraj   = np.empty(nobstimes*ntlm+1)
ytraj   = np.empty(nobstimes*ntlm+1)
ztraj   = np.empty(nobstimes*ntlm+1)

for t in range(nassim):

  #  Advance the truth to the first observation time, obtain observations
  xobs[0], xobs[1], xobs[2] = lor.advance(xt[0], xt[1], xt[2], toff)
  yobs[:,0] = np.matmul(H,xobs) + np.random.normal(0, np.diag(np.sqrt(R)), nobs)
  xbver[:] = xobs[:]

  #  Obtain observations through the window
  for i in range(nobstimes-1):
    xobs[0], xobs[1], xobs[2] = lor.advance(xobs[0], xobs[1], xobs[2], odt)
    yobs[:,i+1] = np.matmul(H,xobs) + np.random.normal(0, np.diag(np.sqrt(R)), nobs)

  #  Step the truth from the previous analysis time to the next one.
  xt[0], xt[1], xt[2] = lor.advance(xt[0], xt[1], xt[2], assim_len)
 
  # step to the next assimilation time
  xb[0], xb[1], xb[2] = lor.advance(xa[0], xa[1], xa[2], toff)

  xi[:] = xb[:]
  niters = 0
  maxiter = 100
  Jold = 2e8
  J = 1e8
  Jold2 = 2e9
  Jmin = 1e9

  alphar = alpha

  while abs(Jold - J) > tol and niters < maxiters and J < Jold*1.1:

    Jold = J
    Jold2 = Jold

    #  Compute the background and observation cost function at first time in window
    innov[:] = yobs[:,0] - np.matmul(H,xi)
    Jb = np.matmul(np.matmul(np.transpose(xi - xb), invB), xi - xb)
    Jy = np.matmul(np.matmul(np.transpose(innov), invR), innov)

    Jbg = 2.0 * np.matmul(invB,xi - xb)
    Jyg = 2.0 * np.matmul(np.matmul(np.transpose(H),invR),innov)

    #  Advance the model forecast through the window, save the output for TLM calculation
    ttraj[0] = 0.0
    xtraj[0] = xi[0]
    ytraj[0] = xi[1]
    ztraj[0] = xi[2]
    for k in range(ntlm*nobstimes):
      ttraj[k+1] = ttraj[k] + tlmdt
      xtraj[k+1], ytraj[k+1], ztraj[k+1] = lor.advance(xtraj[k], ytraj[k], ztraj[k], tlmdt)

    #  Compute the cost function for each of the observation times
    for i in range(nobstimes-1):

      tstep = ntlm*(i+1)
      M = lor.calc_tlm(ttraj[0:tstep], xtraj[0:tstep], ytraj[0:tstep], ztraj[0:tstep])

      xobs[:] = [xtraj[tstep], ytraj[tstep], ztraj[tstep]]

      innov[:] = yobs[:,i+1] - np.matmul(H,xobs)
      Jy  = Jy + np.matmul(np.matmul(np.transpose(innov), invR), innov)
      Jyg = Jyg + 2.0 * np.matmul(np.matmul(np.matmul(np.transpose(M),np.transpose(H)),invR),innov) 

    # total cost function and cost function gradient
    J  = Jb + Jy
    Jg = Jbg - Jyg

    # if the cost function increased, reset X and cut the line-search parameter
    if J > Jold:
#      print('decreasing alpha... ',J,Jold)
      xi[:] = xo[:]
      alphar = alphar/2
      Jold = 2e8; J = 1e8
      adecr = 1
      niters = niters - 1
      continue
    else:
      adecr = 0

    # try to increase alpha if we're past the difficult part
    if adecr < 1 and alphar < alpha:
#      print('increasing alpha...')
      alphar = alphar*1.1
  
    xo[:] = xi[:]

    if niters == 0:  # first iteration do a line search
      xi[:] = xi[:] - alphar*Jg[:]
      Jcgo = Jg[:]

      #  save initial values
      iJb = Jb
      iJy = Jy
      iJ  = J
    else:
      beta = np.matmul(np.transpose(Jg),Jg) / np.matmul(np.transpose(Jgo),Jgo)
      Jcg = Jg[:] + beta*Jcgo[:]
      xi[:] = xi[:] - alphar*Jcg[:]
      Jcgo = Jcg[:]

    Jgo = Jg[:]
    niters = niters + 1

    # save cost function min in case next step is to larger value
    if J < Jmin:
      Jmin = J
      xmin = xi[:]

    print('   ',niters,' cost (J,Jb,Jy)= ',J,Jb,Jy)

  print('assimilation step: ',t)
  print('inital cost (J, Jb, Jy): ', iJ, iJb, iJy)
  print('final cost (J, Jb, Jy) : ', J, Jb, Jy, ' after ', niters, ' iterations')
#  itstats(k) = niters; % use this to tune alpha

  # integrate the IC from start of assimilation window to the analysis time
  xa[0], xa[1], xa[2] = lor.advance(xmin[0], xmin[1], xmin[2], fdvarwin-ctime)

  print('analyzed values:', xa)
  print('verification   :', xt)

  xberr[t,:] = xb[:] - xbver[:]
  xaerr[t,:] = xa[:] - xt[:]

  # compute extended forecast and error
  xf[0], xf[1], xf[2] = lor.advance(xa[0], xa[1], xa[2], fcst_len)
  xe[0], xe[1], xe[2] = lor.advance(xt[0], xt[1], xt[2], fcst_len)

  xferr[t,:] = xf[:] - xe[:]


print('Analysis Error: ',np.sqrt(sum(sum(xaerr[:] * xaerr[:])) / float(nassim*3)))
print('Background Error: ',np.sqrt(sum(sum(xberr[:] * xberr[:])) / float(nassim*3)))
print('Forecast Error: ',np.sqrt(sum(sum(xferr * xferr)) / float(nassim*3)))

time2 = time.time()

print("Total Time:",time2-time1)

    1  cost (J,Jb,Jy)=  107.7870680302132 0.0 107.7870680302132
    1  cost (J,Jb,Jy)=  107.7870680302132 0.0 107.7870680302132
    1  cost (J,Jb,Jy)=  107.7870680302132 0.0 107.7870680302132
    1  cost (J,Jb,Jy)=  107.7870680302132 0.0 107.7870680302132
    1  cost (J,Jb,Jy)=  107.7870680302132 0.0 107.7870680302132
    1  cost (J,Jb,Jy)=  107.7870680302132 0.0 107.7870680302132
    2  cost (J,Jb,Jy)=  101.50900852302755 0.00016445192184380645 101.5088440711057
    2  cost (J,Jb,Jy)=  101.50900852302755 0.00016445192184380645 101.5088440711057
    3  cost (J,Jb,Jy)=  33.21825748078681 0.0009004389861394088 33.217357041800675
    4  cost (J,Jb,Jy)=  29.61014182147193 0.001060266317165899 29.609081555154766
    5  cost (J,Jb,Jy)=  28.06144089608747 0.0011987424376456314 28.060242153649824
    6  cost (J,Jb,Jy)=  27.420204390414703 0.0012990199823933346 27.41890537043231
    7  cost (J,Jb,Jy)=  26.75868412621188 0.0014461443863814565 26.757237981825497
    8  cost (J,Jb,Jy)=  26.4712544